In [1]:
%load_ext autoreload
%autoreload 2

from xml.etree.ElementTree import Element, SubElement, ElementTree
from nxedit.xml_generator.xml_maker import graph_to_xml
from nxedit.xml_generator.utils import calc_tree_pos,pos_to_str_pos
import networkx as nx

import joblib


In [2]:
# node id should begin with Task_****

In [3]:

g=joblib.load("temp/g.bin")

In [4]:
pos=calc_tree_pos(g)
graph_to_xml(g,pos_to_str_pos(pos),"temp/example.xml")